In [ ]:
import numpy as np
from nilearn import image
from nilearn.image import get_data
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from arch.bootstrap import IIDBootstrap
from scipy.stats import kurtosis
import re
import time
import json
import os
#import dask
#from dask.distributed import Client
#from dask import delayed

# Get images and collapsing data

In [ ]:
import glob
import pandas as pd
path= 'Bianca_test_results'
label_path='with_category.csv'
indicative="000"
df_labels=pd.read_csv(label_path,sep=';')
df_labels['id1']=indicative+df_labels['id'].astype(str)

In [ ]:
img_dir = os.listdir("Bianca_test_results")
img_dir.remove('NCC_bianca') 

In [ ]:
img_dir = [st for st in img_dir if st!='.DS_Store']
len(img_dir)

In [ ]:
nx=45;ny=109;nz=91;nt=1
dat_healt, dat_pathologic=getDataFromDirectoryBianca(img_dir,nx,ny,nz,nt)

In [ ]:
print('health neuroimaging: ',len(dat_healt))
print('pathologic neuroimaging: ',len(dat_pathologic))

# Permutations between hemispheres test (HCP)

In [ ]:
# checking the random generation
n=nx*ny*nz
check_rangene = []
for i in range(1000):
    cal = permut(n)
    check_rangene.append(cal)

In [ ]:
print('average of unique values per iteration: ',np.mean(check_rangene))

In [ ]:
resul_permut_helth=[]
resul_permut_pathologic=[]
n=nx*ny*nz

In [ ]:
%%time
for i in np.arange(len(dat_healt)):
    res = rPermut(500,dat_healt[i]['left'],dat_healt[i]['right'],1,n,'rate')
    Resh = {'R':res['R'],'label':dat_healt[i]['label'].tolist(),'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_helth.append(Resh)

In [ ]:
%%time
for i in np.arange(len(dat_pathologic)):
    res = rPermut(500,dat_pathologic[i]['left'],dat_pathologic[i]['right'],1,n,'rate')
    #Rp = res['R']
    #lp = res['label']
    Resp = {'R':res['R'],'label':dat_pathologic[i]['label'].tolist(),'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_pathologic.append(Resp)

In [ ]:
print('health neuroimaging test: ',len(resul_permut_helth))
print('pathologic neuroimaging test: ',len(resul_permut_pathologic))

# Save and load data

In [ ]:
# Save healty
dumped_healty = json.dumps(resul_permut_helth, cls=NumpyEncoder)

with open('resul_permut_helth_Bianca_corrected.json', 'w') as f:
    json.dump(dumped_healty, f)

In [ ]:
# Save patholigic
dumped_pathologic = json.dumps(resul_permut_pathologic, cls=NumpyEncoder)

with open('resul_permut_pathologic_Bianca_corrected.json', 'w') as f:
    json.dump(dumped_pathologic, f)

In [ ]:
# Load data healty
with open('resul_permut_helth_Bianca_corrected.json', 'r') as fp:
    permut_helth = json.load(fp)
res_permut_helth = json.loads(permut_helth) 

In [ ]:
# Load data pathologic
with open('resul_permut_pathologic_Bianca_corrected.json', 'r') as fp:
    permut_pathologic = json.load(fp)
res_permut_pathologic = json.loads(permut_pathologic) 

In [ ]:
len(res_permut_pathologic[0]['R'])

# Comparing results

In [ ]:
distTest(20,5,4,res_permut_helth[0:20],'healty','L1',col='#9addf4')


In [ ]:
distTest(15,5,3,res_permut_pathologic[0:15],'Pathologic','L1',col='#e94975')


In [ ]:
healthy_summary = compareTest(res_permut_helth)


In [ ]:
pathologic_summary = compareTest(res_permut_pathologic)


In [ ]:
thresholdsd=0.01073666
reff=0.01
checkH = checkHCP(res_permut_helth,threshold0,thresholdn,thresholdsd,reff)
checkP = checkHCP(res_permut_pathologic,threshold0,thresholdn,thresholdsd,reff)

In [ ]:
np.unique(checkH, return_counts=True)

In [ ]:
np.unique(checkP, return_counts=True)